In [2]:
# Conserve code au cas où / sinon code valide situé 2ème cellule

import requests
from bs4 import BeautifulSoup
import json
import time
import re

def extraire_informations_pays(url_pays):
    """
    Extrait les informations essentielles sur les médailles d'un pays
    à partir de sa page sur olympics-statistics.com
    """
    try:
        response = requests.get(url_pays)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extraction du nom du pays
        h1_title = soup.select_one('h1')
        nom_pays = ""
        if h1_title and "Olympic medals from" in h1_title.text:
            nom_pays = h1_title.text.replace("Olympic medals from", "").strip()
        
        # Extraction de la période des Jeux
        periode_element = soup.select_one('h3')
        periode = periode_element.text.strip() if periode_element else ""
        
        # Extraction du nombre de médailles, jeux, sports et épreuves
        h2_stats = soup.select_one('h2')
        stats = {}
        if h2_stats:
            stats_text = h2_stats.text.strip()
            # Utilisation de regex pour extraire les chiffres et catégories
            medals_match = re.search(r'(\d+)\s+Medals', stats_text)
            games_match = re.search(r'(\d+)\s+Games', stats_text)
            sports_match = re.search(r'(\d+)\s+Sports', stats_text)
            events_match = re.search(r'(\d+)\s+Events', stats_text)
            
            if medals_match: stats['nb_medals'] = int(medals_match.group(1))
            if games_match: stats['nb_games'] = int(games_match.group(1))
            if sports_match: stats['nb_sports'] = int(sports_match.group(1))
            if events_match: stats['nb_events'] = int(events_match.group(1))
        
        # Extraction des médailles par type (or, argent, bronze)
        medal_teaser = soup.select_one('div.rnd.teaser')
        medal_counts = {
            'summer': 0,
            'winter': 0,
            'gold': 0,
            'silver': 0,
            'bronze': 0,
            'total': 0
        }
        
        if medal_teaser:
            # Extraction des saisons (Summer/Winter)
            # Trouver tous les spans avec la classe "mal"
            all_spans = medal_teaser.select('span.mal')
            
            # Trouver les divs qui contiennent "Summer" et "Winter"
            all_divs = medal_teaser.find_all('div')
            
            summer_count = 0
            winter_count = 0
            
            # Parcourir les divs pour trouver "Summer" et "Winter"
            for i, div in enumerate(all_divs):
                div_text = div.get_text(strip=True)
                if "Summer" in div_text:
                    # Trouver le span.mal correspondant
                    if i < len(all_spans):
                        summer_count = int(all_spans[i].get_text(strip=True))
                        medal_counts['summer'] = summer_count
                elif "Winter" in div_text:
                    # Trouver le span.mal correspondant
                    if i < len(all_spans):
                        winter_count = int(all_spans[i].get_text(strip=True))
                        medal_counts['winter'] = winter_count
            
            # Si aucun texte Summer/Winter n'est trouvé, on peut essayer une approche alternative
            if medal_counts['summer'] == 0 and medal_counts['winter'] == 0:
                for span in all_spans:
                    next_text = span.find_next_sibling(text=True)
                    if next_text and "Summer" in next_text:
                        medal_counts['summer'] = int(span.get_text(strip=True))
                    elif next_text and "Winter" in next_text:
                        medal_counts['winter'] = int(span.get_text(strip=True))
            
            # Extraction des médailles d'or, d'argent et de bronze
            medal_divs = medal_teaser.select('div > div.the-medal')
            for medal_div in medal_divs:
                medal_type = medal_div.get('data-medal')
                # Trouver le span.mal associé
                parent_div = medal_div.parent
                count_span = parent_div.select_one('span.mal')
                if count_span:
                    count = int(count_span.get_text(strip=True))
                    if medal_type == '1':
                        medal_counts['gold'] = count
                    elif medal_type == '2':
                        medal_counts['silver'] = count
                    elif medal_type == '3':
                        medal_counts['bronze'] = count
            
            # Calculer le total
            medal_counts['total'] = medal_counts['gold'] + medal_counts['silver'] + medal_counts['bronze']
        
        # Construction du dictionnaire final
        pays_info = {
            'nom': nom_pays,
            'periode': periode,
            'statistiques': stats,
            'medailles': medal_counts
        }
        
        return pays_info
    
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations pour {url_pays}: {str(e)}")
        return None

def collecter_pays():
    """
    Collecte les URLs de tous les pays présents sur la page des nations
    """
    url_nations = "https://olympics-statistics.com/nations"
    try:
        response = requests.get(url_nations)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        pays_links = []
        nation_cards = soup.select('a.card.nation')
        
        for card in nation_cards:
            pays_info = {}
            
            # URL
            href = card.get('href')
            url_complete = f"https://olympics-statistics.com{href}" if href else ""
            
            # Nom du pays
            pays_div = card.select_one('div.bez')
            pays_nom = pays_div.get_text(strip=True) if pays_div else ""
            
            # Image du drapeau
            flag_img = card.select_one('img')
            flag_src = flag_img.get('src') if flag_img else ""
            flag_url = f"https://olympics-statistics.com{flag_src}" if flag_src else ""
            
            # Code pays (extrait de l'URL)
            pays_id = None
            if href:
                id_match = re.search(r'/nation/[^/]+/(\d+)', href)
                if id_match:
                    pays_id = id_match.group(1)
            
            pays_info = {
                'id': pays_id,
                'nom': pays_nom,
                'url': url_complete,
                'drapeau': flag_url
            }
            
            pays_links.append(pays_info)
        
        return pays_links
    
    except Exception as e:
        print(f"Erreur lors de la collecte des pays: {str(e)}")
        return []

def main():
    # Collecte des pays
    print("Collecte des pays...")
    pays_links = collecter_pays()
    print(f"{len(pays_links)} pays trouvés.")
    
    # Collecte des informations détaillées pour chaque pays
    resultats = []
    
    # Option pour tester avec un nombre limité de pays
    limite_test = None  # Mettre None pour traiter tous les pays, ou un nombre pour limiter
    pays_a_traiter = pays_links[:limite_test] if limite_test else pays_links
    
    for i, pays in enumerate(pays_a_traiter):
        print(f"Traitement du pays {i+1}/{len(pays_a_traiter)}: {pays['nom']}")
        
        # Extraction des informations du pays
        pays_info = extraire_informations_pays(pays['url'])
        
        if pays_info:
            # Ajout des informations de base
            pays_info['id'] = pays['id']
            pays_info['drapeau'] = pays['drapeau']
            resultats.append(pays_info)
            
            # Pause pour éviter de surcharger le serveur
            time.sleep(1)
    
    # Sauvegarde des résultats dans un fichier JSON
    with open('olympics_nations.json', 'w', encoding='utf-8') as f:
        json.dump(resultats, f, ensure_ascii=False, indent=4)
    
    print(f"Terminé. Les données de {len(resultats)} pays ont été sauvegardées dans 'olympics_nations.json'.")

if __name__ == "__main__":
    main()

Collecte des pays...
178 pays trouvés.
Traitement du pays 1/178: Afghanistan
Traitement du pays 2/178: Albania
Traitement du pays 3/178: Algeria
Traitement du pays 4/178: Argentina
Traitement du pays 5/178: Armenia
Traitement du pays 6/178: Australia
Traitement du pays 7/178: Australia / New Zealand (Australasia)
Traitement du pays 8/178: Austria
Traitement du pays 9/178: Austria / United States of Amerika
Traitement du pays 10/178: Azerbaijan
Traitement du pays 11/178: Bahamas
Traitement du pays 12/178: Bahrain
Traitement du pays 13/178: Barbados
Traitement du pays 14/178: Belarus
Traitement du pays 15/178: Belgium
Traitement du pays 16/178: Bermuda
Traitement du pays 17/178: Bohemia
Traitement du pays 18/178: Bohemia / Great Britain
Traitement du pays 19/178: Botswana
Traitement du pays 20/178: Brazil
Traitement du pays 21/178: Bulgaria
Traitement du pays 22/178: Burkina-Faso
Traitement du pays 23/178: Burundi
Traitement du pays 24/178: Cameroon
Traitement du pays 25/178: Canada
Trai

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Traitement du pays 72/178: Iceland
Traitement du pays 73/178: Independent Olympic Participants
Traitement du pays 74/178: India
Traitement du pays 75/178: Individual Neutral Athletes
Traitement du pays 76/178: Individual Olympic Athletes
Traitement du pays 77/178: Indonesia
Traitement du pays 78/178: Iran
Traitement du pays 79/178: Iraq
Traitement du pays 80/178: Ireland
Traitement du pays 81/178: Israel
Traitement du pays 82/178: Italy
Traitement du pays 83/178: Jamaica
Traitement du pays 84/178: Japan 
/ Korea
Traitement du pays 85/178: Japan


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Traitement du pays 86/178: Jordan
Traitement du pays 87/178: Kazakhstan
Traitement du pays 88/178: Kenya
Traitement du pays 89/178: Kosovo
Traitement du pays 90/178: Kuwait
Traitement du pays 91/178: Kyrghyzstan
Traitement du pays 92/178: Latvia
Traitement du pays 93/178: Lebanon
Traitement du pays 94/178: Liechtenstein
Traitement du pays 95/178: Lithuania
Traitement du pays 96/178: Luxembourg
Traitement du pays 97/178: Luxembourgh / France
Traitement du pays 98/178: Malaysia
Traitement du pays 99/178: Mauritius
Traitement du pays 100/178: Mexico
Traitement du pays 101/178: Moldova
Traitement du pays 102/178: Mongolia
Traitement du pays 103/178: Montenegro
Traitement du pays 104/178: Morocco
Traitement du pays 105/178: Mozambique
Traitement du pays 106/178: Namibia
Traitement du pays 107/178: Netherlands
Traitement du pays 108/178: Netherlands / France
Traitement du pays 109/178: Netherlands Antilles
Traitement du pays 110/178: New Zealand
Traitement du pays 111/178: Niger
Traitement d

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Traitement du pays 143/178: Spain
Traitement du pays 144/178: Sri Lanka
Traitement du pays 145/178: Sudan
Traitement du pays 146/178: Suriname
Traitement du pays 147/178: Sweden
Traitement du pays 148/178: Sweden / Denmark
Traitement du pays 149/178: Switzerland
Traitement du pays 150/178: Syria
Traitement du pays 151/178: Tajikistan
Traitement du pays 152/178: Tanzania
Traitement du pays 153/178: Thailand
Traitement du pays 154/178: Thessalonika
Traitement du pays 155/178: Togo
Traitement du pays 156/178: Tonga
Traitement du pays 157/178: Trinidad & Tobago
Traitement du pays 158/178: Tunisia
Traitement du pays 159/178: Turkey
Traitement du pays 160/178: Turkmenistan
Traitement du pays 161/178: Uganda
Traitement du pays 162/178: Ukraine
Traitement du pays 163/178: United Arab Emirates
Traitement du pays 164/178: United States of America
Traitement du pays 165/178: United States of America / Cuba
Traitement du pays 166/178: United States of America / France
Traitement du pays 167/178: U

In [4]:
# 
import requests
from bs4 import BeautifulSoup
import json
import time
import re

def extraire_informations_pays(url_pays):
    """
    Extrait les informations essentielles sur les médailles d'un pays
    à partir de sa page sur olympics-statistics.com
    """
    try:
        # Spécifier l'encodage UTF-8 explicitement pour éviter les problèmes de caractères
        response = requests.get(url_pays)
        response.encoding = 'utf-8'  # Définir explicitement l'encodage
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraction du nom du pays
        h1_title = soup.select_one('h1')
        nom_pays = ""
        if h1_title and "Olympic medals from" in h1_title.text:
            nom_pays = h1_title.text.replace("Olympic medals from", "").strip()
        
        # Extraction de la période des Jeux
        periode_element = soup.select_one('h3')
        periode = periode_element.text.strip() if periode_element else ""
        
        # Extraction du nombre de médailles, jeux, sports et épreuves
        h2_stats = soup.select_one('h2')
        stats = {}
        if h2_stats:
            stats_text = h2_stats.text.strip()
            # Utilisation de regex pour extraire les chiffres et catégories
            medals_match = re.search(r'(\d+)\s+Medals', stats_text)
            games_match = re.search(r'(\d+)\s+Games', stats_text)
            sports_match = re.search(r'(\d+)\s+Sports', stats_text)
            events_match = re.search(r'(\d+)\s+Events', stats_text)
            
            if medals_match: stats['nb_medals'] = int(medals_match.group(1))
            if games_match: stats['nb_games'] = int(games_match.group(1))
            if sports_match: stats['nb_sports'] = int(sports_match.group(1))
            if events_match: stats['nb_events'] = int(events_match.group(1))
        
        # Initialisation du comptage de médailles
        medal_counts = {
            'summer': 0,
            'winter': 0,
            'gold': 0,
            'silver': 0,
            'bronze': 0,
            'total': 0
        }
        
        # Extraction des médailles
        medal_teaser = soup.select_one('div.rnd.teaser')
        if medal_teaser:
            # Pour déboguer, affichons le contenu HTML
            # print(medal_teaser.prettify())
            
            # Méthode plus robuste pour extraire Summer et Winter
            summer_winter_divs = medal_teaser.find_all('div', recursive=False)
            if len(summer_winter_divs) >= 1:
                first_div = summer_winter_divs[0]
                
                # Extraire les médailles d'été
                summer_span = first_div.select_one('span.mal')
                if summer_span:
                    try:
                        medal_counts['summer'] = int(summer_span.text.strip())
                    except ValueError:
                        pass
                
                # Vérifier si le div contient "Summer"
                if "Summer" in first_div.text:
                    # C'est bien le div pour l'été
                    pass
                elif "Winter" in first_div.text:
                    # Oups, c'était l'hiver, pas l'été
                    medal_counts['winter'] = medal_counts['summer']
                    medal_counts['summer'] = 0
                
                # S'il y a un second div pour Winter
                if len(summer_winter_divs) >= 2:
                    second_div = summer_winter_divs[1]
                    winter_span = second_div.select_one('span.mal')
                    if winter_span and "Winter" in second_div.text:
                        try:
                            medal_counts['winter'] = int(winter_span.text.strip())
                        except ValueError:
                            pass
            
            # Maintenant extraire les médailles par type 
            medal_divs = medal_teaser.select('div > div.the-medal')
            for medal_div in medal_divs:
                medal_type = medal_div.get('data-medal')
                if not medal_type:
                    continue
                
                # Trouver le span.mal associé
                parent_div = medal_div.parent
                count_span = parent_div.select_one('span.mal')
                if count_span:
                    try:
                        count = int(count_span.text.strip())
                        if medal_type == '1':
                            medal_counts['gold'] = count
                        elif medal_type == '2':
                            medal_counts['silver'] = count
                        elif medal_type == '3':
                            medal_counts['bronze'] = count
                    except ValueError:
                        pass
            
            # Calculer le total des médailles
            medal_counts['total'] = medal_counts['gold'] + medal_counts['silver'] + medal_counts['bronze']
        
        # Construction du dictionnaire final
        pays_info = {
            'nom': nom_pays,
            'periode': periode,
            'statistiques': stats,
            'medailles': medal_counts
        }
        
        return pays_info
    
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations pour {url_pays}: {str(e)}")
        return None

def collecter_pays():
    """
    Collecte les URLs de tous les pays présents sur la page des nations
    """
    url_nations = "https://olympics-statistics.com/nations"
    try:
        response = requests.get(url_nations)
        response.encoding = 'utf-8'  # Définir explicitement l'encodage
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pays_links = []
        nation_cards = soup.select('a.card.nation')
        
        for card in nation_cards:
            pays_info = {}
            
            # URL
            href = card.get('href')
            url_complete = f"https://olympics-statistics.com{href}" if href else ""
            
            # Nom du pays
            pays_div = card.select_one('div.bez')
            pays_nom = pays_div.get_text(strip=True) if pays_div else ""
            
            # Image du drapeau
            flag_img = card.select_one('img')
            flag_src = flag_img.get('src') if flag_img else ""
            flag_url = f"https://olympics-statistics.com{flag_src}" if flag_src else ""
            
            # Code pays (extrait de l'URL)
            pays_id = None
            if href:
                id_match = re.search(r'/nation/[^/]+/(\d+)', href)
                if id_match:
                    pays_id = id_match.group(1)
            
            pays_info = {
                'id': pays_id,
                'nom': pays_nom,
                'url': url_complete,
                'drapeau': flag_url
            }
            
            pays_links.append(pays_info)
        
        return pays_links
    
    except Exception as e:
        print(f"Erreur lors de la collecte des pays: {str(e)}")
        return []

def main():
    # Collecte des pays
    print("Collecte des pays...")
    pays_links = collecter_pays()
    print(f"{len(pays_links)} pays trouvés.")
    
    # Collecte des informations détaillées pour chaque pays
    resultats = []
    
    # Option pour tester avec un nombre limité de pays
    limite_test = None  # Mettre None pour traiter tous les pays, ou un nombre pour limiter
    pays_a_traiter = pays_links[:limite_test] if limite_test else pays_links
    
    for i, pays in enumerate(pays_a_traiter):
        print(f"Traitement du pays {i+1}/{len(pays_a_traiter)}: {pays['nom']}")
        
        # Extraction des informations du pays
        pays_info = extraire_informations_pays(pays['url'])
        
        if pays_info:
            # Ajout des informations de base
            pays_info['id'] = pays['id']
            pays_info['drapeau'] = pays['drapeau']
            resultats.append(pays_info)
            
            # Pause pour éviter de surcharger le serveur
            time.sleep(1)
    
    # Sauvegarde des résultats dans un fichier JSON avec l'encodage UTF-8
    with open('olympics_nations.json', 'w', encoding='utf-8') as f:
        json.dump(resultats, f, ensure_ascii=False, indent=4)
    
    print(f"Terminé. Les données de {len(resultats)} pays ont été sauvegardées dans 'olympics_nations.json'.")

if __name__ == "__main__":
    main()

Collecte des pays...
178 pays trouvés.
Traitement du pays 1/178: Afghanistan
Traitement du pays 2/178: Albania
Traitement du pays 3/178: Algeria
Traitement du pays 4/178: Argentina
Traitement du pays 5/178: Armenia
Traitement du pays 6/178: Australia
Traitement du pays 7/178: Australia / New Zealand (Australasia)
Traitement du pays 8/178: Austria
Traitement du pays 9/178: Austria / United States of Amerika
Traitement du pays 10/178: Azerbaijan
Traitement du pays 11/178: Bahamas
Traitement du pays 12/178: Bahrain
Traitement du pays 13/178: Barbados
Traitement du pays 14/178: Belarus
Traitement du pays 15/178: Belgium
Traitement du pays 16/178: Bermuda
Traitement du pays 17/178: Bohemia
Traitement du pays 18/178: Bohemia / Great Britain
Traitement du pays 19/178: Botswana
Traitement du pays 20/178: Brazil
Traitement du pays 21/178: Bulgaria
Traitement du pays 22/178: Burkina-Faso
Traitement du pays 23/178: Burundi
Traitement du pays 24/178: Cameroon
Traitement du pays 25/178: Canada
Trai

In [10]:
import json
import pandas as pd
import numpy as np

# Importation du dictionnaire pays_dict pour la correspondance codes ISO
pays_dict = {
    'afghanistan': 'af',
    'albania': 'al',
    'algeria': 'dz',         
    'argentina': 'ar',
    'armenia': 'am',
    'australia': 'au',
    'austria': 'at',
    'azerbaijan': 'az',
    'bahamas': 'bs',
    'bahrain': 'bh',
    'barbados': 'bb',
    'belarus': 'by',
    'belgium': 'be',
    'bermuda': 'bm',
    'botswana': 'bw',
    'brazil': 'br',
    'bulgaria': 'bg',
    'burkina-faso': 'bf',
    'burundi': 'bi',
    'cameroon': 'cm',         
    'canada': 'ca',
    'cape verde': 'cv',
    'chile': 'cl',
    'china': 'cn',
    'chinese taipei': 'tw',
    'colombia': 'co',
    'costa rica': 'cr',
    'cote divoire': 'ci',
    'croatia': 'hr',
    'cuba': 'cu',
    'cyprus': 'cy',
    'czech republic': 'cz',
    'denmark': 'dk',
    'djibouti': 'dj',
    'dominican': 'dm',
    'dominican republic': 'do',
    'ecuador': 'ec',
    'egypt': 'eg',
    'eritrea': 'er',
    'estonia': 'ee',
    'ethiopia': 'et',
    'fiji': 'fj',
    'finland': 'fi',
    'france': 'fr',
    'gabon': 'ga',
    'georgia': 'ge',
    'germany': 'de',
    'ghana': 'gh',
    'great britain': 'gb',
    'greece': 'gr',
    'grenada': 'gd',
    'guatemala': 'gt',
    'guyana': 'gy',
    'haiti': 'ht',
    'hong kong': 'hk',
    'hungary': 'hu',
    'iceland': 'is',
    'india': 'in',
    'indonesia': 'id',
    'iran': 'ir',
    'iraq': 'iq',
    'ireland': 'ie',
    'israel': 'il',
    'italy': 'it',
    'jamaica': 'jm',
    'japan': 'jp',
    'jordan': 'jo',         
    'kazakhstan': 'kz',
    'kenya': 'ke',
    'kosovo': 'xk',
    'kuwait': 'kw',
    'kyrgyzstan': 'kg',
    'latvia': 'lv',
    'lebanon': 'lb',
    'liechtenstein': 'li',
    'lithuania': 'lt',
    'luxembourg': 'lu',       
    'malaysia': 'my',
    'mauritius': 'mu',
    'mexico': 'mx',
    'moldova': 'md',
    'mongolia': 'mn',
    'montenegro': 'me',
    'morocco': 'ma',         
    'mozambique': 'mz',
    'namibia': 'na',
    'netherlands': 'nl',
    'new zealand': 'nz',
    'niger': 'ne',
    'nigeria': 'ng',
    'north korea': 'kp',     
    'north-macedonia': 'mk',
    'norway': 'no',
    'pakistan': 'pk',
    'panama': 'pa',
    'paraguay': 'py',
    'peru': 'pe',
    'philippines': 'ph',
    'poland': 'pl',
    'portugal': 'pt',
    'puerto rico': 'pr',     
    'qatar': 'qa',        
    'romania': 'ro',
    'russia': 'ru',
    'saint lucia': 'lc',     
    'samoa': 'ws',
    'san marino': 'sm',   
    'saudi arabia': 'sa',     
    'senegal': 'sn',
    'serbia': 'rs',
    'singapore': 'sg',
    'slovakia': 'sk',
    'slovenia': 'si',
    'south africa': 'za',
    'south korea': 'kr',
    'spain': 'es',
    'sri lanka': 'lk',
    'sudan': 'sd',
    'suriname': 'sr',
    'sweden': 'se',
    'switzerland': 'ch',
    'syria': 'sy',           
    'tajikistan': 'tj',      
    'tanzania': 'tz',
    'thailand': 'th',
    'togo': 'tg',
    'tonga': 'to',
    'trinidad & tobago': 'tt',
    'tunisia': 'tn',         
    'turkey': 'tr',
    'turkmenistan': 'tm',
    'uganda': 'ug',          
    'ukraine': 'ua',
    'united arab emirates': 'ae', 
    'united states of america': 'us',
    'united states': 'us',
    'uruguay': 'uy',       
    'uzbekistan': 'uz',      
    'venezuela': 've',
    'vietnam': 'vn',
    'virgin islands': 'vi',
    'zambia': 'zm',
    'zimbabwe': 'zw'
}

# Création d'un dictionnaire inversé pour recherche par code
codes_pays_dict = {v: k for k, v in pays_dict.items()}

# Chargement des données
try:
    with open('olympics_nations.json', 'r', encoding='utf-8') as f:
        nations = json.load(f)
    print(f"Données chargées : {len(nations)} nations trouvées")
except FileNotFoundError:
    print("Fichier olympics_nations.json introuvable. Exécutez d'abord le script principal.")
    nations = []

# Conversion en DataFrame pour faciliter le nettoyage
df = pd.DataFrame(nations)

# Aperçu initial
print("\nAperçu avant nettoyage:")
print(df.head())
print(f"Colonnes: {df.columns.tolist()}")

# Fonction pour nettoyer les données des nations
def nettoyer_data_nations():
    if len(df) == 0:
        print("Pas de données à nettoyer.")
        return df
    
    # Créer des colonnes pour les médailles et statistiques
    # 1. Médailles par type
    df['or'] = df['medailles'].apply(lambda x: x.get('gold', 0) if isinstance(x, dict) else 0)
    df['argent'] = df['medailles'].apply(lambda x: x.get('silver', 0) if isinstance(x, dict) else 0)
    df['bronze'] = df['medailles'].apply(lambda x: x.get('bronze', 0) if isinstance(x, dict) else 0)
    df['total_medailles'] = df['medailles'].apply(lambda x: x.get('total', 0) if isinstance(x, dict) else 0)
    
    # 2. Médailles par saison
    df['medailles_ete'] = df['medailles'].apply(lambda x: x.get('summer', 0) if isinstance(x, dict) else 0)
    df['medailles_hiver'] = df['medailles'].apply(lambda x: x.get('winter', 0) if isinstance(x, dict) else 0)
    
    # # 3. Statistiques
    # df['nb_jeux'] = df['statistiques'].apply(lambda x: x.get('nb_games', 0) if isinstance(x, dict) else 0)
    # df['nb_sports'] = df['statistiques'].apply(lambda x: x.get('nb_sports', 0) if isinstance(x, dict) else 0)
    # df['nb_epreuves'] = df['statistiques'].apply(lambda x: x.get('nb_events', 0) if isinstance(x, dict) else 0)
    
    # Correction des noms de pays et ajout du code ISO
    df['nom'] = df['nom'].fillna('Unknown')
    df['pays_code'] = ''
    
    # Ajouter le code pays en fonction du nom
    for idx in df.index:
        nom_pays = df.loc[idx, 'nom'].lower().strip()
        # Pour gérer le cas des États-Unis qui peut avoir plusieurs notations
        if 'united states' in nom_pays or nom_pays == 'usa':
            df.loc[idx, 'pays_code'] = 'us'
        elif nom_pays in pays_dict:
            df.loc[idx, 'pays_code'] = pays_dict[nom_pays]
        # Parfois le nom du pays inclut des infos additionnelles comme "Great Britain (GBR)"
        elif '(' in nom_pays:
            nom_base = nom_pays.split('(')[0].strip().lower()
            if nom_base in pays_dict:
                df.loc[idx, 'pays_code'] = pays_dict[nom_base]
            # Essayer avec le code entre parenthèses si présent
            elif len(nom_pays.split('(')) > 1:
                code_potentiel = nom_pays.split('(')[1].replace(')', '').lower()
                if len(code_potentiel) <= 3 and code_potentiel in codes_pays_dict:
                    df.loc[idx, 'pays_code'] = code_potentiel
    
    # Normalisation des noms de pays (première lettre en majuscule pour chaque mot)
    df['nom'] = df['nom'].apply(lambda x: x.title() if isinstance(x, str) else x)
    
    # Trouver les pays sans code ISO détecté
    pays_sans_code = df[df['pays_code'] == '']['nom'].tolist()
    if pays_sans_code:
        print(f"\nATTENTION: Pays sans code ISO détecté: {', '.join(pays_sans_code)}")
    
    # Suppression des colonnes imbriquées (maintenant extraites)
    df_clean = df.drop(columns=['medailles', 'statistiques'])
    
    # Statistiques post-nettoyage
#     print("\nStatistiques après nettoyage:")
#     print(f"Nombre total de nations: {len(df_clean)}")
#     print(f"Nombre total de médailles: {df_clean['total_medailles'].sum()}")
#     print(f"Médailles d'or: {df_clean['or'].sum()}")
#     print(f"Médailles d'argent: {df_clean['argent'].sum()}")
#     print(f"Médailles de bronze: {df_clean['bronze'].sum()}")
#     print(f"Médailles d'été: {df_clean['medailles_ete'].sum()}")
#     print(f"Médailles d'hiver: {df_clean['medailles_hiver'].sum()}")
    
#     # Top 5 des nations avec le plus de médailles
#     print("\nTop 5 des nations avec le plus de médailles:")
#     top_5 = df_clean.sort_values('total_medailles', ascending=False).head(5)
#     for idx, row in top_5.iterrows():
#         print(f"{row['nom']} ({row['pays_code'].upper()}): {row['total_medailles']} médailles (Or: {row['or']}, Argent: {row['argent']}, Bronze: {row['bronze']})")
    
    
    # Conversion du DataFrame en liste de dictionnaires (format original)
    nations_clean_dict = df_clean.to_dict(orient='records')

    # Enregistrement dans un fichier JSON avec une indentation pour meilleure lisibilité
    with open('olympics_nations_clean.json', 'w', encoding='utf-8') as f:
        json.dump(nations_clean_dict, f, ensure_ascii=False, indent=4)

    print(f"\nDonnées nettoyées enregistrées au format JSON dans 'olympics_nations_clean.json'")
    print(f"Nombre de nations enregistrées: {len(nations_clean_dict)}")

    # Aperçu du premier élément
    if len(nations_clean_dict) > 0:
        print("\nAperçu de la première nation au format JSON:")
        print(json.dumps(nations_clean_dict[0], ensure_ascii=False, indent=2))
    
    return df_clean

# Exécution du nettoyage
df_clean = nettoyer_data_nations()

# Aperçu final
print("\nAperçu après nettoyage:")
print(df_clean.head())

Données chargées : 178 nations trouvées

Aperçu avant nettoyage:
           nom      periode statistiques  \
0  Afghanistan  2008 - 2012           {}   
1      Albania         2024           {}   
2      Algeria  1984 - 2024           {}   
3    Argentina  1924 - 2024           {}   
4      Armenia  1996 - 2024           {}   

                                           medailles  id  \
0  {'summer': 2, 'winter': 0, 'gold': 0, 'silver'...   1   
1  {'summer': 2, 'winter': 0, 'gold': 0, 'silver'...   3   
2  {'summer': 20, 'winter': 0, 'gold': 7, 'silver...   4   
3  {'summer': 80, 'winter': 0, 'gold': 22, 'silve...  10   
4  {'summer': 22, 'winter': 0, 'gold': 2, 'silver...  11   

                                             drapeau  
0  https://olympics-statistics.com/media/flagge/a...  
1  https://olympics-statistics.com/media/flagge/a...  
2  https://olympics-statistics.com/media/flagge/d...  
3  https://olympics-statistics.com/media/flagge/a...  
4  https://olympics-statistics.com